# Notebook para hacer una predicción de preguntas de Reddit usando un dataset de entrenamiento de Quora



In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Not connected to a GPU
Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


Cargamos ambos datasets y los concateno en una misma lista

In [2]:
#Importación de las librerías NLTK y Gensim

import nltk
import gensim
import pandas as pd
import re
import random
nltk.download('all')

#Cargo el dataset de las preguntas de Quora
total_questions_quora_df = pd.read_csv('train.csv')


#Selecciono las preguntas insinceras y sinceras de Quora establencienco un tamaño de 20.000 para cada muestra
tamanio=20000

insincere_questionsdf=total_questions_quora_df.loc[total_questions_quora_df['target'] == 1].sample(tamanio)
sincere_questionsdf=total_questions_quora_df.loc[total_questions_quora_df['target'] == 0].sample(tamanio)

#Las unimos en una misma lista
questions_Quora=insincere_questionsdf['question_text'].append(sincere_questionsdf['question_text']).tolist()
tags_Quora = insincere_questionsdf['target'].append(sincere_questionsdf['target']).tolist()

#Selecciono las preguntas de reddit (en principio 30.000)
reddit_df = pd.read_csv('reddit.csv').sample(20000)
reddit_questions=reddit_df['question_text'].tolist()
reddit_tags=reddit_df['target'].tolist()

#Unimos en un mismo dataset los datos de las dos fuentes
questions_QuoraReddit=questions_Quora+reddit_questions
labels_QuoraReddit=tags_Quora+reddit_tags

#Proceso los textos
data = []
data_labels = []
for i in range(len(questions_QuoraReddit)): 
    questions_QuoraReddit[i]=questions_QuoraReddit[i].lower()
    questions_QuoraReddit[i]=re.sub('[^a-zA-Z\']', ' ', questions_QuoraReddit[i])
    data.append(questions_QuoraReddit[i]) 
    data_labels.append(labels_QuoraReddit[i])
    
data_labels = ['SINCERE' if x==0 else 'INSINCERE' for x in data_labels]



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

Se utiliza un tfidf Vectorizer de todo el conjunto y separamos entrenamiento (Quora) y dataset (Reddit)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Usamos tfidVectorizer y seleccionamos Quora para train y Reddit para validar
tamanioQuora=tamanio*2

tfidfvectorizer = TfidfVectorizer(analyzer= 'word')
X = tfidfvectorizer.fit_transform(data)
X = X.toarray()
X_train=X[0:tamanioQuora]
X_test=X[tamanioQuora:len(data)]
y_train=data_labels[0:tamanioQuora]
y_test=data_labels[tamanioQuora:len(data)]

Utilizo un clasificador de regresión lineal, que es el que mejor resultado dio en el Notebook relacionado con Quora.

In [4]:
#Clasificador con regresión logística para ver la bondad del predictor 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression()
##Entrenamiento
log_model = classifier.fit(X=X_train, y=y_train)
#Clasificación
y_pred = log_model.predict(X_test)

#Se calculan las métricas de evaluación
lm = metrics.classification_report(y_test, y_pred, labels=list(set(data_labels)))

print(lm)

from collections import Counter
print(Counter(y_test))

              precision    recall  f1-score   support

   INSINCERE       0.34      0.89      0.49      1279
     SINCERE       0.99      0.88      0.93     18721

    accuracy                           0.88     20000
   macro avg       0.67      0.89      0.71     20000
weighted avg       0.95      0.88      0.90     20000

Counter({'SINCERE': 18721, 'INSINCERE': 1279})


El proceso es muy bueno a la hora de asegurar la sinceridad de las preguntas sinceras, pero falla al detectar las insinceras. Por ello vamos a hacer un breve análisis  LDA de las preguntas insinceras

In [3]:
#Funciones auxiliares
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.models.phrases import Phraser
from gensim.models import Phrases

#Cargo todas las stopwords y añado algunas típicas del inglés no incluidas en nltk
stopwords = nltk.corpus.stopwords.words('english') + ['ve', 'hadn', 'll', 'didn', 'isn', 'doesn', 'hasn', 'n\'t' ]

#La función get_wn_pos traduce el formato de PoS del término según el pos-tagger al formato de PoS de Wordnet
def get_wn_pos(pos):
    if re.match(r'^N',pos):
        wn_pos = 'n'
    elif re.match(r'^V',pos):
        wn_pos = 'v'
    else:
        wn_pos = 'n' #En inglés, los lemas de términos que no son verbos ni nombres se obtienen como si fueran
                        #nombres
    return wn_pos

#La función wnlemmatize lematiza el término con una etiqueta PoS según el lematizador de Wordnet
def wnlemmatize(t,postag):
    lemma = ""
    #Definición del lematizador
    lem = WordNetLemmatizer()
    #Si el candidato es monopalabra, se obtiene el lema con el lematizador de WordNet según su PoS
    if ' ' not in t:
        lemma = lem.lemmatize(t,get_wn_pos(postag[0][1]))
    #Si el candidato es multipalabra, obtenemos su lema como si fuera un nombre, aplicando el lematizador de WordNet
    else:
        lemma = lem.lemmatize(t,'n')
    return lemma

#Categorías a excluir para un buen candidato    
no_pos_in = ['DT', 'IN', 'PRP', 'CC', 'CD','MD', 'VBG', 'VBD', 'RP', 'RB']

    #Método para evaluar la idoneidad de una palabra o una combinación de palabras
def good_candidate(t,postag):
    v = False
    #Si es multipalabra
    if ' ' in t:
        tl = t.split(' ') #Generamos una lista de tokens
        #el token inicial y el token final deben ser alfabéticos y no pueden estar en la lista de stopwords..
        if re.match("^[a-z]+.*", tl[0]) and re.match("^[a-z]+.*", tl[-1]) and \
           tl[0] not in stopwords and tl[1] not in stopwords:
            #... ni su PoS puede estar en la lista no_pos_in
            if postag[0][1] not in no_pos_in and postag[-1][1] not in no_pos_in:
                v = True
    #Si es monopalabra
    else:
        #debe ser alfabético, y no estar en la lista de stopwords
        if t not in stopwords and re.match("^[a-z]+.*", t):
            #y su PoS no puede estar en la lista no_pos_in
            if postag[0][1] not in no_pos_in:
                v = True
    return v

In [4]:
from gensim.models.phrases import Phraser
from gensim.models import Phrases
from nltk import pos_tag, word_tokenize
#Lista con todas las palabras de todas las preguntas
text_stream = [word_tokenize(str(d).lower()) for d in reddit_df['question_text'].tolist()]
#Genero el modelo de phrases
phrases = Phrases(text_stream, min_count=1, threshold=2, delimiter=b' ')

#Me quedo con las insinceras
insincere_questionsdf=reddit_df.loc[reddit_df['target'] == 1]
insincere_questions_list = insincere_questionsdf['question_text'].tolist()

postag = {}
def transform_sentence(sentence):
    #Obtenemos los phrases según el modelo de detección de phrases que ha aprendido
    sentence_phrases = phrases[word_tokenize(sentence.lower())]
    #Quitamos los signos de puntuación de los phrases
    phrases_stripped = [st.strip('".,;:-():!?-‘’ ') for st in sentence_phrases if re.match("^[a-z]+.*", st)]
    #Hacemos etiquetaje de PoS de los phrases y lo guardamos en un diccionario (postag) 
    for ps in phrases_stripped:
        postag[ps] = nltk.pos_tag(word_tokenize(ps))
    document_terms = [tps for tps in phrases_stripped if good_candidate(tps,postag[tps]) == True]
    #Lematizamos los phrases con el lematizador de Wordnet
    phrases_lemmatized = [wnlemmatize(ps,postag[ps]) for ps in document_terms]
    return phrases_lemmatized

transformed_sentences = [transform_sentence(ss) for ss in insincere_questions_list]

print(transformed_sentences[:5])

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['question', 'jehovah', 'witness', 'answer'], ['indian women', 'high', 'demand', 'men', 'contribution'], ['conservative', 'politics', 'liberal', 'current', 'climate', 'thing'], ['article', 'israeli', 'atrocity', 'expose', 'belligerent', 'occupier', 'support'], ['assassination', 'politician', 'conservative', 'liberal']]


In [6]:
dictionary = gensim.corpora.Dictionary(transformed_sentences)


#Creo el BOW para cada documento
bow_corpus = [dictionary.doc2bow(doc) for doc in transformed_sentences]


lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 3, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)


#Vamos a ver las palabras que hay en cada topic y el peso que tienen
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.010*"people" + 0.009*"indian" + 0.008*"india" + 0.007*"gay" + 0.006*"white" + 0.005*"country" + 0.005*"black" + 0.004*"know" + 0.004*"trump" + 0.004*"jew"


Topic: 1 
Words: 0.010*"people" + 0.010*"muslim" + 0.009*"men" + 0.008*"woman" + 0.008*"girl" + 0.005*"liberal" + 0.004*"white people" + 0.004*"state" + 0.004*"child" + 0.004*"time"


Topic: 2 
Words: 0.011*"trump" + 0.009*"woman" + 0.008*"american" + 0.006*"people" + 0.006*"hindu" + 0.005*"right" + 0.004*"chinese" + 0.003*"hate" + 0.003*"liberal" + 0.003*"european"




Vamos a hacer un análisis del Tfidfvectorizer

In [8]:
#Importar los métodos de la librería NLTK que lematizan según Wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.models.phrases import Phraser
from gensim.models import Phrases
#Ponemos las preguntas insinceras en una lista
insincere_questionsdf_reddit=reddit_df.loc[reddit_df['target'] == 1]
sincere_questionsdf_reddit=reddit_df.loc[reddit_df['target'] == 0]

insincere_questions_list_reddit = insincere_questionsdf_reddit['question_text'].tolist()

#Ponemos las sinceras en otra lista
sincere_questions_list_reddit = sincere_questionsdf_reddit['question_text'].tolist()

#Lista con todas las palabras de todas las preguntas
text_stream = [word_tokenize(str(d).lower()) for d in reddit_df['question_text'].tolist()]
#print(text_stream[:10])

#Convertimos las insinceras en un documento
terms_insincere_doc = " ".join(insincere_questions_list_reddit)
#Convertimos los sinceros en un documento
terms_sincere_doc = " ".join(sincere_questions_list_reddit)

#Creamos la colección de documentos con las sinceras y las insinceras
docs_col = [terms_insincere_doc, terms_sincere_doc]

#Obtengo la lista de phrases de todo el texto
phrases = Phrases(text_stream, min_count=1, threshold=2, delimiter=b' ')

#Método para transformar los documentos para posteriormente introducir en el tdIdfVectorizer
def transform_doc(doc):
    #Obtenemos los phrases según el modelo de detección de phrases que ha aprendido
    phraser = Phraser(phrases)
    doc_phrases = phraser[word_tokenize(doc.lower())]
    #Quitamos los signos de puntuación de los phrases
    text_phrases_stripped = [dp.strip('".,;:-():!?-‘’ ') for dp in doc_phrases if re.match("^[a-z]+.*", dp) ]
    #Hacemos etiquetaje de PoS de los phrases y lo guardamos en un diccionario (postag) 
    postag = {}
    for tps in text_phrases_stripped:
        postag[tps] = nltk.pos_tag(word_tokenize(tps))
    #Obtenemos los phrases que son términos, aplicando el método good_candidate y teniendo en cuenta 
    #su etiquetaje PoS.
    document_terms = [tps for tps in text_phrases_stripped if good_candidate(tps,postag[tps]) == True]
    #Lematizamos los términos con el lematizador de Wordnet
    terms_lemmatized = [wnlemmatize(dt,postag[dt]) for dt in document_terms]
    #Concatenamos los constituyentes de un término multipalabra con '_'
    terms_lemmatized_and_unified2display = [tlu.replace(' ','_') for tlu in terms_lemmatized ]
    return " ".join(terms_lemmatized_and_unified2display)

transf_col = [transform_doc(d) for d in docs_col]

#Importar la librería con los métodos para hacer la vectorización
from sklearn.feature_extraction.text import TfidfVectorizer

#Definimos el vectorizador. Su analyzer busca términos de tipo 'word'. 

vectorizador = TfidfVectorizer(
    analyzer= 'word',
    )

vec_fit = vectorizador.fit_transform(transf_col)

#Tomamos el array correspondiente a las preguntas insinceras
ta = vec_fit.toarray()[0]

#La relación feature name-valor de tf.idf se expresa en forma de tupla. El primer elemento de la tupla es el
#feature name y el segundo elemento es su valor de tf.idf en el documento N, en este caso el documento 0.
#Estas tuplas se van poniendo en una lista (tfidf_tuples) para que luego puedan ordenadarse de mayor a menor valor

tfidf_tuples = []

feature_names = vectorizador.get_feature_names()

for i in range(0, len(feature_names)):
    tfidf_tuples.append((feature_names[i], ta[i]))
    
#Se ordenan las tuplas
sorted_tuples = sorted(tfidf_tuples, key=lambda tup: tup[1], reverse=True)

labels = ['Term', 'TfIdf']

#Se crea un dataframe a partir del cual se construirá la tabla
df3 = pd.DataFrame.from_records(sorted_tuples, columns=labels)

#Construcción y visualización de la tabla
print ("")
print ("Valores de TfIdf de los términos en las preguntas insinceras") 
print ("")
print (df3. head(15))


Valores de TfIdf de los términos en las preguntas insinceras

        Term     TfIdf
0     people  0.363833
1      woman  0.282588
2      trump  0.240200
3     muslim  0.204877
4     indian  0.169553
5      india  0.151891
6   american  0.148359
7        men  0.148359
8    liberal  0.130697
9       girl  0.123632
10   country  0.120100
11     black  0.116568
12      hate  0.116568
13     hindu  0.116568
14     white  0.109503


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
